# Table of Contents
1. Introduction
2. Models
    * [U-Net3](https://arxiv.org/ftp/arxiv/papers/2004/2004.08790.pdf)
    * [U-2-Net](https://arxiv.org/pdf/2005.09007v1.pdf)    
    * [U-Net++](https://arxiv.org/abs/1807.10165)
    * [U-Net](https://link.springer.com/chapter/10.1007%2F978-3-319-24574-4_28)
    * [ResUNet](https://arxiv.org/abs/1904.00592)
    
3. Results 


## Introduction
**Melanoma**
* an uncommon skin cancer ([ACS, 2020](https://www.cancer.org/cancer/melanoma-skin-cancer/about/what-is-melanoma.html))
    * skin cancer is most prevalent cancer ([SCF,2020](https://www.skincancer.org/skin-cancer-information/skin-cancer-facts/))
* responsible for 75% of skin cancer deaths ([Peterson, 2016](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4998683/))
* 100,000 new melanoma cases will be diagnosed in 2020 ([ACS, 2020](https://www.cancer.org/content/dam/cancer-org/research/cancer-facts-and-statistics/annual-cancer-facts-and-figures/2020/cancer-facts-and-figures-2020.pdf)). 
* 7,000 people will die from melanoma in 2020 ([Trager, 2020](https://practicaldermatology.com/articles/2020-apr/update-biomarkers-for-guiding-treatment-in-early-stage-melanoma#:~:text=Although%20it%20accounts%20for%20only,the%20end%20of%20this%20year.&text=The%20American%20Joint%20Committee%20on,stage%20I%20through%20IV%20disease.)).
* If caught early, most melanomas can be cured with minor surgery ([ACS, 2020](https://www.cancer.org/cancer/melanoma-skin-cancer/treating/surgery.html#:~:text=When%20melanoma%20is%20diagnosed%20by,numb%20it%20before%20the%20excision.)).
* Currently, dermatologists evaluate every one of a patient's moles to identify outlier lesions or “ugly ducklings” that are most likely to be melanoma ([SCF, 2020](https://www.skincancer.org/skin-cancer-information/melanoma/melanoma-warning-signs-and-images/)).
* Early and accurate detection, aided by artificial intelligence utilizing "contextual" info within each patient, can make treatment more effective and positively impact millions of people ([Nature, 2020](https://www.nature.com/articles/d41586-020-00847-2)).

**This Competition Notebook**
* melanoma is identified in [images of skin lesions from the ISIC Archive](https://www.kaggle.com/c/siim-isic-melanoma-classification/data),the largest publicly available collection of quality-controlled dermoscopic images of skin lesions.
* The most widely successfull lesion detection classification algorithms (e.g. [U-Net3](https://arxiv.org/ftp/arxiv/papers/2004/2004.08790.pdf), [U-2-Net](https://arxiv.org/pdf/2005.09007v1.pdf), [U-Net++](https://arxiv.org/abs/1807.10165), [U-Net](https://link.springer.com/chapter/10.1007%2F978-3-319-24574-4_28), [ResUNet](https://arxiv.org/abs/1904.00592)) are utilized to automatically detect and label melanoma in the images.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import sys
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
from PIL import Image
import pydicom
import tensorflow as tf


#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
os.chdir('/kaggle/input/')
dpath='/kaggle/input/siim-isic-melanoma-classification/'

In [ ]:
train_csv=pd.read_csv('./siim-isic-melanoma-classification/train.csv')
train_csv

In [ ]:
# read tfrec
tfrd=os.path.join(dpath,'./tfrecords/')
tfr=tf.data.TFRecordDataset(tfrd)

In [ ]:
type(tfrd+('./test06-687.tfrec'))

In [ ]:
# Create a dictionary describing the features.
image_feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

In [ ]:
def _parse_image_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = tfr.map(_parse_image_function)
parsed_image_dataset

In [ ]:
# dcm to jpg and info
dcm_path='/kaggle/input/siim-isic-melanoma-classification/train/'
outpath='/kaggle/working/dcm_jpg/'
images_path=os.listdir(dcm_path)

In [ ]:
# read dcm
dcdir=os.path.join(dpath,'train/')
dc1=pydicom.dcmread(dcdir+'ISIC_0074311.dcm')

In [ ]:
dc1

In [ ]:
# Create list of first 20 jpg
tjd='/kaggle/input/siim-isic-melanoma-classification/jpeg/train/'
ims=[]
limit=20
processed=0
for i in os.listdir(tjd):
    impath=os.path.join(tjd,i)
    if os.path.isfile(impath):
        j=Image.open(impath)
        ims.append(j)
    processed += 1
    if processed > limit:
        break
ims

In [ ]:
# Plot first 20 jpg
fig=plt.figure(figsize=(15,10))
columns=5;rows=4
for i in range(1,columns*rows+1):
    img=ims[i]
    fig.add_subplot(rows,columns,i)
    plt.imshow(img)
plt.show